In [27]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv

from ML.DTGenerated import DTGenerated
import pickle

In [28]:
from generateTestingData.useGeneratedData import makeDataUsable, splitTrainTest, NewPredicter

In [29]:
df = makeDataUsable(read_csv("../data/testGenerateData.csv"),4, 3)
print(df.shape)
df.head()

(570, 30)


,Time,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,...,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4,COL
0,3.0,50.0,50.0,50.0,50.000,100.0,100.0,100.0,100.0000,1.546,...,0.000,-0.202,-0.000,-0.001,-0.001,0.027,0.006,0.001,0.000,0.0
1,4.0,50.0,50.0,50.0,50.000,100.0,100.0,100.0,100.0000,0.661,...,-0.001,-0.000,-0.001,-0.001,0.000,0.006,0.001,0.000,-0.001,0.0
2,5.0,50.0,50.0,50.0,50.000,100.0,100.0,100.0,100.0000,4.117,...,-0.000,-0.001,-0.001,0.000,0.000,0.001,0.000,-0.001,0.000,0.0
3,6.0,50.0,50.0,50.0,50.000,100.0,100.0,100.0,100.0000,0.865,...,0.000,-0.001,0.000,0.000,0.000,0.000,-0.001,0.000,0.000,0.0
4,7.0,50.0,50.0,50.0,5.322,100.0,100.0,100.0,83.3296,3.704,...,0.004,0.000,0.000,0.000,-0.001,-0.001,0.000,0.000,0.005,0.0


In [30]:
df["COL"].value_counts()

0.0    501
1.0     69
Name: COL, dtype: int64

In [31]:
# df = pd.concat([df[df["COL"] == False].sample(80, random_state=1), df[df["COL"] == True]])

In [32]:
data = df.copy().drop(columns=df.columns[0], axis=1)
trainX, trainY, testX, testY = splitTrainTest(data)

splitting at 119.


In [33]:
trainX.head(3)

,TTC1,TTC2,TTC3,TTC4,DTO1,DTO2,DTO3,DTO4,JERK1,JERK2,...,asX3,asX4,asY1,asY2,asY3,asY4,asZ1,asZ2,asZ3,asZ4
551,-0.471,50.000,50.0,50.0,-0.0066,-0.0090,44.7045,27.2004,0.753,0.025,...,0.016,0.104,-0.000,-0.000,-0.310,-0.011,-0.000,-0.001,0.016,0.008
39,-0.007,-2.275,50.0,50.0,-0.0090,-0.0091,0.9216,0.7991,1.445,1.525,...,0.002,0.004,-0.002,0.000,0.001,-0.000,0.003,0.000,-0.001,-0.001
510,33.527,50.000,50.0,50.0,10.4939,100.0000,39.8169,100.0000,0.588,4.672,...,-0.003,-0.004,-0.001,-0.309,0.001,-0.001,-0.001,-0.030,-0.015,0.002


In [34]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

In [35]:
xgb_clf = NewPredicter(XGBClassifier(n_estimators=100, random_state=1))
xgb_trainX = xgb_clf.preProcess(trainX.copy())
xgb_testX = xgb_clf.preProcess(testX.copy())

xgb_trainY, xgb_testY = trainY.to_numpy(), testY.to_numpy()

Scaler is fitted


In [36]:
xgb_clf.fit(xgb_trainX, xgb_trainY)

In [37]:
len(xgb_testX[0])
xgb_pred = xgb_clf.predict(xgb_testX)
xgb_score = xgb_clf.getScore(xgb_testY, xgb_pred)

Total: 30, Collisions: 11
Accuracy: 0.7333333333333333, Precision: 0.6666666666666666, Recall: 0.5454545454545454, F1: 0.6
Confusion matrix:
[[16  3]
 [ 5  6]]


TODO - need more data, what is radar?

In [38]:
# model_score = f"{score[0][0]}-{score[0][1]}-{score[1][0]}-{score[1][1]}"
model_score = f"{xgb_score[0][0]}-{xgb_score[0][1]}-{xgb_score[1][0]}-{xgb_score[1][1]}"
model_score

'16-3-5-6'

In [39]:
# xgb_clf.saveModel("xgb_2", accuracy=model_score)